This notebook provides a baseline loan applications classifier based on Scikit-Learn's Random Forest. It doesn't include complex feature engineering, but provides high F1 and ROC AUC score on 5-fold cross-validation.

In [8]:
import pandas as pd
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score, make_scorer

import hashlib

In [9]:
# We drop unique ID and borrower's organization name right away as they are useless.
# We also drop ChgOffDate, ChgOffPrinGr because they can directly tell us that the loan is charged-off.
# 'ApprovalDate', 'ApprovalFY', 'DisbursementDate' are dropped to make the model time-independent.
df = pd.read_csv("/Users/sentinel/Developer/SBAnational.csv",low_memory=False).drop(columns=['LoanNr_ChkDgt', 'Name', 'ChgOffDate', 'ChgOffPrinGr','ApprovalDate', 'ApprovalFY', 'DisbursementDate'])
len_data = len(df)
df

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv
0,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,84,4,2.0,0,0,1,0,N,Y,"$60,000.00",$0.00,P I F,"$60,000.00","$48,000.00"
1,NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,60,2,2.0,0,0,1,0,N,Y,"$40,000.00",$0.00,P I F,"$40,000.00","$32,000.00"
2,BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,180,7,1.0,0,0,1,0,N,N,"$287,000.00",$0.00,P I F,"$287,000.00","$215,250.00"
3,BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,60,2,1.0,0,0,1,0,N,Y,"$35,000.00",$0.00,P I F,"$35,000.00","$28,000.00"
4,ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,240,14,1.0,7,7,1,0,N,N,"$229,000.00",$0.00,P I F,"$229,000.00","$229,000.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899159,UPPER ARLINGTON,OH,43221,JPMORGAN CHASE BANK NATL ASSOC,IL,451120,60,6,1.0,0,0,1,0,0,N,"$70,000.00",$0.00,P I F,"$70,000.00","$56,000.00"
899160,COLUMBUS,OH,43221,JPMORGAN CHASE BANK NATL ASSOC,IL,451130,60,6,1.0,0,0,1,0,Y,N,"$85,000.00",$0.00,P I F,"$85,000.00","$42,500.00"
899161,SANTA MARIA,CA,93455,"RABOBANK, NATIONAL ASSOCIATION",CA,332321,108,26,1.0,0,0,1,0,N,N,"$300,000.00",$0.00,P I F,"$300,000.00","$225,000.00"
899162,HONOLULU,HI,96830,BANK OF HAWAII,HI,0,60,6,1.0,0,0,1,0,N,Y,"$75,000.00",$0.00,CHGOFF,"$75,000.00","$60,000.00"


In [10]:
# Let's convert the strings styled as '$XXXX.XX' to float values
money_cols = ['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']

# Here, we remove the comma from the values.
for col in money_cols:
  df[col] = [float(val[1:].replace(',', '')) for val in df[col].values]

In [11]:
# Let's check our data for missing values and fill NAs with mode. The mode is the value that appears the most.
for col in df.drop(columns=['MIS_Status']).columns:
  if df[col].isna().any():
    df[col] = df[col].fillna(df[col].mode().iloc[0])

In [12]:
# We have many columns with Object dtype; let's apply one hot encoding
# (if the number of unique values is relatively small)
# or hashing if there are many uniques
# The only exception is MIS_Status (our target) variable: it is 'PIF' if the loan is returned
# and 'CHGOFF' if the borrower had a debt
cols_to_drop = []

for col in df.drop(columns=['MIS_Status']).columns:
  if df[col].dtype == 'object':
    print(f'Column {col} has {df[col].nunique()} values among {len_data}')

    if df[col].nunique() < 25:
      print(f'One-hot encoding of {col}')
      one_hot_cols = pd.get_dummies(df[col])
      for ohc in one_hot_cols.columns:
        df[col + '_' + ohc] = one_hot_cols[ohc]
    else:
      print(f'Hashing of {col}')
      df[col + '_hash'] = df[col].apply(lambda row: int(hashlib.sha1((col + "_" + str(row)).encode('utf-8')).hexdigest(), 16) % len_data)

    cols_to_drop.append(col)

Column City has 32581 values among 899164
Hashing of City
Column State has 51 values among 899164
Hashing of State
Column Bank has 5802 values among 899164
Hashing of Bank
Column BankState has 56 values among 899164
Hashing of BankState
Column RevLineCr has 18 values among 899164
One-hot encoding of RevLineCr
Column LowDoc has 8 values among 899164
One-hot encoding of LowDoc


In [13]:
# Converting target variable from string to binary
df = df.drop(columns=cols_to_drop)

df['Defaulted'] = [1 if app == 'CHGOFF' else 0 for app in df.MIS_Status.values]
df = df.drop(columns=['MIS_Status'])

In [14]:
# Finally, our data looks like this:
df

,Zip,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,DisbursementGross,...,RevLineCr_`,LowDoc_0,LowDoc_1,LowDoc_A,LowDoc_C,LowDoc_N,LowDoc_R,LowDoc_S,LowDoc_Y,Defaulted
0,47711,451120,84,4,2.0,0,0,1,0,60000.0,...,0,0,0,0,0,0,0,0,1,0
1,46526,722410,60,2,2.0,0,0,1,0,40000.0,...,0,0,0,0,0,0,0,0,1,0
2,47401,621210,180,7,1.0,0,0,1,0,287000.0,...,0,0,0,0,0,1,0,0,0,0
3,74012,0,60,2,1.0,0,0,1,0,35000.0,...,0,0,0,0,0,0,0,0,1,0
4,32801,0,240,14,1.0,7,7,1,0,229000.0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899159,43221,451120,60,6,1.0,0,0,1,0,70000.0,...,0,0,0,0,0,1,0,0,0,0
899160,43221,451130,60,6,1.0,0,0,1,0,85000.0,...,0,0,0,0,0,1,0,0,0,0
899161,93455,332321,108,26,1.0,0,0,1,0,300000.0,...,0,0,0,0,0,1,0,0,0,0
899162,96830,0,60,6,1.0,0,0,1,0,75000.0,...,0,0,0,0,0,0,0,0,1,1


In [15]:
# The dataset is quite imbalanced: the amount of non-defaulted loans is 5x of that of defaulted ones
print(df.Defaulted.value_counts())

0    741606
1    157558
Name: Defaulted, dtype: int64


In [16]:
# Let's fit and cross-validate a balanced random forest; first, divide the data to X and Y...
X = df.drop(columns=['Defaulted'])
Y = df.Defaulted

In [17]:
# ...and apply stratified 5-fold validation
rfc = RandomForestClassifier(class_weight='balanced', random_state=42)
f1_scorer = make_scorer(f1_score)
auc_scorer = make_scorer(roc_auc_score)
cross_validate(rfc, X, Y, cv=StratifiedKFold(random_state=42, shuffle=True), scoring=['f1_weighted', 'roc_auc'], n_jobs=-1, verbose=10)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


[CV] START .....................................................................
[CV] START .....................................................................
[CV] START .....................................................................
[CV] START .....................................................................
[CV] START .....................................................................
[CV] END .... f1_weighted: (test=0.937) roc_auc: (test=0.967) total time= 4.7min
[CV] END .... f1_weighted: (test=0.937) roc_auc: (test=0.967) total time= 4.7min


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  4.8min remaining:  7.2min
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:  4.8min remaining:  3.2min


[CV] END .... f1_weighted: (test=0.938) roc_auc: (test=0.969) total time= 4.7min
[CV] END .... f1_weighted: (test=0.937) roc_auc: (test=0.968) total time= 4.7min
[CV] END .... f1_weighted: (test=0.936) roc_auc: (test=0.968) total time= 4.7min


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  4.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  4.8min finished


{'fit_time': array([263.28509903, 261.82141495, 262.51544189, 262.35877299,
        263.10355902]),
 'score_time': array([20.44506884, 20.92335415, 20.47501206, 20.38254404, 20.25798082]),
 'test_f1_weighted': array([0.93628599, 0.93659654, 0.93754751, 0.93694861, 0.93726374]),
 'test_roc_auc': array([0.96772686, 0.96739675, 0.96889811, 0.96719507, 0.96840178])}

As we can see, this model provides average F1 of 0.94 and average ROC AUC of 0.97, which is close to 1 and, thus, efficient to detect potentially risky loan applications. To improve the baseline solution, we can:
- dive deeper into the problem and create new informative features;
- apply more sophisticated methods, such as boosting or deep learning;
- try to use oversampling techniques.